# Referral partner analysis

In [ ]:
# Imports

import os
import pandas as pd
import numpy as np
import datetime
import calendar
from dateutil.relativedelta import relativedelta
import re

## >> Input required

##### Define filepaths

In [ ]:
input_file = '.../log.csv' # Filepath to log
output_folder = '...' # Filepath to folder where the analysis will be saved
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

##### Define analysis dates

In [ ]:
# Define report end date
report_end = datetime.date(2018, 5, 31) # Enter year, month, day

# Define analysis window (months)
window = 13

## Set up

In [ ]:
# Useful references

ordered_categories = ["contact",
                      "early_help_assessment_start",
                      "early_help_assessment_completion",
                      "referral",
                      "assessment_start",
                      "assessment_authorised",
                      "s47",
                      "icpc",
                      "cin_start",
                      "cin_end",
                      "cpp_start",
                      "cpp_end",
                      "lac_start",
                      "lac_end"]
referral_sources = {
    '1': 'Individual',
    '2': 'Schools',
    '3': 'Health services',
    '4': 'Housing',
    '5': 'LA services',
    '6': 'Police',
    '7': 'Other legal agency',
    '8': 'Other',
    '9': 'Anonymous',
    '10': 'Unknown'    
}

In [ ]:
# Functions to use during analysis


def central_event_to_other_events(central_event, dataframe):
    '''
    Returns a dataframe matching a central event with all other events that happened to a child
    The dataframe includes a days elapsed column counting the days between central event and the other event
    Removes the duplicates: a central event cannot be matched to the same central event
    '''
    # Add column with event sequence to track the order in which the events took place
    dataframe['child_event_sequence'] = dataframe.sort_values(['Date','Type']).groupby(['CUID']).cumcount() + 1
    
    # Get list of all central events we want to assess in relation to other events
    central_events = dataframe[dataframe.Type == central_event]

    # Merge the dataframe to the central_events: this will pair each central event to any event that happened to the same child
    # i.e. there will be multiple lines for each combination of [central event + another event]
    central_event_to_other_event = central_events.merge(dataframe, how='left', on='CUID')

    # Calculate the number of days between central event and its paired event
    central_event_to_other_event["days_elapsed"] = (central_event_to_other_event.Date_y - central_event_to_other_event.Date_x) / np.timedelta64(1, 'D')

    # Remove rows where central event = paired event
    same = (central_event_to_other_event.child_event_sequence_x == central_event_to_other_event.child_event_sequence_y)
    central_event_to_other_event = central_event_to_other_event[~(same)]
    
    return central_event_to_other_event


def apply_duration_filter(data, name, event_type):
    '''
    Counts days to closest next event
    '''
    criteria_next = data[(data.Type_y == event_type) & (data.days_elapsed>=0)]
    df = criteria_next.groupby(["CUID","child_event_sequence_x"]).agg({"days_elapsed": 'min'}).reset_index()
    df.rename(columns={"child_event_sequence_x": "child_event_sequence", "days_elapsed": 'days_to_{}'.format(name)}, inplace=True)     
    return df


def start_end_date(date, period):
    '''
    Returns start and end date based on a given date and given period (months)
    The given date must be a Timestamp
    The end date will be the last day of the month prior to the given date
    The start date will be the first day of the month after calculating (end date - period)
    '''
    # If given date is already at the end of the month
    if date.day == calendar.monthrange(date.year, date.month)[1]:
        end_date = date
        start_date = end_date + relativedelta(months=-(period-1))
        start_date = start_date.replace(day=1)
    # If given date is not at the end of the month
    else:
        end_date_intermediate = date.replace(day=1)
        end_date = end_date_intermediate - np.timedelta64(1, 'D')
        start_date = end_date_intermediate + relativedelta(months=-period)
    try:
        start_date = start_date.replace(hour = 0, minute = 0, second = 0)
    except:
        print('No time in start date')
    try:
        end_date = end_date.replace(hour = 23, minute = 59, second = 59)
    except:
        print('No time in end date')
    # Turn into pd-compatible format
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    return start_date, end_date


def above_within_below(series, lower_bound, upper_bound):
    '''
    For a given series, calculates whether its values are above, within, below the given lower_bounds and upper_bounds
    '''
    values = []
    for data, low, high in zip(series, lower_bound, upper_bound):
        if data < low:
            value = 'below'
        elif data > high:
            value = 'above'
        elif (data >= low) and (data <= high):
            value = 'within'
        else:
            value = np.nan
        values.append(value)
    return values

## Analysis

In [ ]:
# Create base dataframe: all_events

# Load input_file
log = pd.read_csv(input_file)
# Set categories of events to make sure we order them correctly if two events happen on the same day
# e.g. Contact comes before Referral if same date
log.Type = log.Type.astype('category')
log.Type.cat.set_categories([c for c in ordered_categories if c in log.Type.unique()], inplace=True, ordered=True)
log.Date = pd.to_datetime(log.Date)

# Drop rows where the Contact Source was not matched correctly at Annex A cleaner stage: we can't use them
contacts_not_matched = log[log['Contact Source'] == 'not matched'].CUID.count()
total_contacts = log[log.Type=='contact']['CUID'].count()
print('There were {} contacts that were not matched out of {} contacts ({}%)'.format(contacts_not_matched, total_contacts, round(contacts_not_matched/total_contacts*100, 2)))
log = log[~(log['Contact Source']=='not matched')]

# Drop rows without a Type, as we don't know what they are
log = log[~(log.Type.isnull())]

# Create a copy of log named all_events - we will use it throughout the notebook
all_events = log.copy()

# Only keep the columns we need for the analysis
all_events = all_events[['CUID', 'Date', 'Type', 'Source', 'Referral Source', 'Contact Source']]
all_events.head()

### 1. Create Contact to other events DataFrame

- Input: all_events
- Output: DataFrame of all contacts and their number of days to next contact, next referral and next assessment

This is done in four steps: 
- Step 1: Create a DataFrame matching each contact to all the other events that happened to the same child
- Step 2: Create a DataFrame with a single line per contact with number of days from that contact to the nearest contact, referral and assessment
- Step 3: Add a few columns to identify referral date, assessment date and deduce contacts outcomes
- Step 4: Tidy up DataFrame

In [ ]:
# Set up

# Choose the 'central' event - all day counts will be based on this event. In this case: contacts.
central_event = 'contact'

# Write list of events which days will be counted in relation to central event
# Syntax: {short name: event type}
events = {'contact':'contact',
          'referral': 'referral',
          'assessment':'assessment_start'}

In [ ]:
# Step 1 - Match contacts to all other events

contacts_to_next_event = central_event_to_other_events(central_event, all_events)



# Step 2 - Count days from contact to nearest events mentioned in {events}

contacts_day_count = all_events[all_events.Type == central_event]
for event in events.keys():
    to_next = apply_duration_filter(contacts_to_next_event, event, events[event])
    contacts_day_count = contacts_day_count.merge(to_next, how="left", on=["CUID","child_event_sequence"])
    

    
# Step 3 - Add extra columns to define referral date, assessment date and deduce contacts outcomes

# Convert day counts into Timedelta
for event in events:
    contacts_day_count['days_to_{}'.format(event)] = contacts_day_count['days_to_{}'.format(event)].apply(lambda x: pd.Timedelta(x, unit='D'))

# Calculate referral and assessment dates
contacts_day_count['referral_date'] = contacts_day_count.Date + contacts_day_count.days_to_referral
contacts_day_count['assessment_date'] = contacts_day_count.Date + contacts_day_count.days_to_assessment

# Match referrals to contacts: must be same CUID, same referral date, same Referral Source
referrals = all_events[(all_events.Type == 'referral')&(all_events['Referral Source'] != 'not matched')][['CUID', 'Date', 'Referral Source']]
referrals['referral_matched'] = True # This will help track which contacts where successfully matched
referrals.rename(columns={'Date': 'referral_date', 'Referral Source':'Contact Source'}, inplace=True)
contacts_day_count.drop('Referral Source', axis=1, inplace=True)
contacts_day_count = contacts_day_count.merge(referrals, how='left', on=['CUID', 'referral_date', 'Contact Source'])
total_referrals = all_events[all_events.Type == 'referral'].shape[0]
matched_referrals = contacts_day_count[contacts_day_count.referral_matched == True].shape[0]
print('{} referrals were matched to a contact, out of a total of {} referrals ({}%)'.format(matched_referrals, total_referrals, round(matched_referrals/total_referrals*100,2)))

# Create outcomes columns: led_to_referral, led_to_assessment and contact_only

# Referral
# A contact led to referral if:
#    1 - A referral was successfully matched to the contact CUID, date and referral source (referral_matched == True)
#    2 - The referral happened within 7 days of the contact
#    3a - There is no further contact
#    3b - OR if there is a further contact, it happens after the referral

referral_criteria = ((contacts_day_count['referral_matched'] == True) 
                     & ((contacts_day_count['referral_date'] - contacts_day_count['Date'])<= np.timedelta64(7, 'D')) 
                     & (contacts_day_count['days_to_contact'].isnull() 
                           | (contacts_day_count['days_to_referral'] < contacts_day_count['days_to_contact']))) 


# Assessment 
# A contact led to assessment if:
#    1 - The assessment happened within 7 days of the referral
#    2a - There is no further contact
#    2b - OR if there is a further contact, it happens after the assessment
assessment_criteria = ((contacts_day_count['assessment_date'] - contacts_day_count['referral_date']<= np.timedelta64(7, 'D')) 
                     & (contacts_day_count['days_to_contact'].isnull() 
                           | (contacts_day_count['days_to_assessment'] < contacts_day_count['days_to_contact']))) 

# Create columns based on criteria
contacts_day_count['led_to_referral'] = referral_criteria
contacts_day_count['led_to_assessment'] = assessment_criteria
contacts_day_count['contact_only'] = (contacts_day_count.led_to_referral == False) & (contacts_day_count.led_to_assessment == False)



# Step 4 - Tidy up

# Simplify Annex A contact source categories, e.g. 5C becomes LA services
contacts_day_count['agg_contact_source'] = contacts_day_count['Contact Source'].apply(lambda x: re.findall(r'(\d{1,2})[A-Z]?', x)[0])
contacts_day_count['agg_contact_source'] = contacts_day_count['agg_contact_source'].map(referral_sources)

# Add year_month columns for contacts, referrals and assessments
contacts_day_count['year_month'] = contacts_day_count.Date.apply(lambda x: str(x)[:7])
contacts_day_count['r_year_month'] = contacts_day_count.referral_date.apply(lambda x: str(x)[:7])
contacts_day_count['a_year_month'] = contacts_day_count.assessment_date.apply(lambda x: str(x)[:7])

contacts_day_count.head()

### 2. Create "master" Dataframe counting Contact, Referrals and Assessments
- Input: contacts_day_count
- Output: DataFrame with following columns:
    - Referral partner (agg_contact_source)
    - Year-Month (year_month)
    - Number of total contacts (total_contacts)
    - Number of contacts NFA (contact_only)
    - Number of contacts that reached referral (led_to_referral)
    - Number of contacts that reached assessment (led_to_assessment)
    - Number of total referrals (total_referrals)
    - Number of total assessments (total_assessments)


This is done in 2 steps:
- Step 1: Count contacts per outcome per source
- Step 2: Count referral and assessments

In [ ]:
# Set up

# Define dates of report analysis
start_date, end_date = start_end_date(report_end, window)
print('The report will start on {} and end on {}'.format(start_date, end_date))

In [ ]:
# Step 1 - Count contacts per outcome (contact only, led to referral, led to assessment) and per source

# Get subset with right time window
contact_outcomes = contacts_day_count[(contacts_day_count.Date >= start_date) & (contacts_day_count.Date <= end_date)]

# Calculate number of contacts per outcome per contact source
contact_outcomes = contact_outcomes.groupby(['agg_contact_source', 'year_month']).agg({'CUID': 'count', 'contact_only': 'sum', 'led_to_referral': 'sum', 'led_to_assessment': 'sum'}).reset_index()
contact_outcomes.rename(columns={'CUID': 'total_contacts'}, inplace=True)


# Step 2 - Count number of referrals and assessments per month per contact source

master = contact_outcomes.copy()
for event in ['referral', 'assessment']:
    grouped = contacts_day_count[contacts_day_count['led_to_{}'.format(event)]==True][['CUID', 'agg_contact_source', '{}_year_month'.format(event[0])]]
    grouped = grouped.groupby(['agg_contact_source', '{}_year_month'.format(event[0])]).agg({'CUID':'count'}).reset_index()
    grouped.rename(columns={'CUID': 'total_{}s'.format(event), '{}_year_month'.format(event[0]):'year_month'}, inplace=True)
    master = master.merge(grouped, how='left', on=['agg_contact_source', 'year_month'])

master.head()

### 3. Create partner, total and comparative dfs and save to Excel
- Input: master
- Output: Excel spreadsheet with the following tabs:
    - Step 1 - One tab per contact source ('{contact source name}') with #contacts, #referrals, #assessments, #contacts_only, #led_to_referral, #led_to_assessment
    - Step 2 - Three tabs ('Comparative contacts/referrals/assessments') comparing contacts, referrals and assessments across contact sources, including a lower and higher standard deviation bounds
    - Step 3 - One tab ('Contact only') counting volume and proportion of contacts that did not lead to further action, across contact sources, for the final month only
    - Step 4 - One tab ('Total') that recaps total #contacts, #referrals, #assessments and standard deviation bounds for all three

In [ ]:
# Set up

# Set up ExcelWriter to store the different tables in separate tabs in a unique Excel spreadsheet
tables = pd.ExcelWriter(os.path.join(output_folder, 'referral_partner_analysis.xlsx'), engine='xlsxwriter')

# Determine final month of analysis (useful for a few dfs)
final_month = master.year_month.max()

In [ ]:
# Step 1 - Data specific to each contact source

for source in master.agg_contact_source.unique():
    df = master[master.agg_contact_source == source]
    
    # Save to excel
    df.to_excel(tables, sheet_name='{}'.format(source), index=False)
    

# Step 2 - Compare across contact sources on contacts, referrals and assessments

for event in ['contacts', 'referrals', 'assessments']:
    df = master[['agg_contact_source', 'year_month', 'total_{}'.format(event)]]
    df = df.pivot(index='agg_contact_source', columns='year_month', values='total_{}'.format(event))
    df['upper_std'] = df.mean(axis = 1) + df.std(axis = 1) 
    df['lower_std'] = df.mean(axis = 1) - df.std(axis = 1)
    df['{}-situation'.format(final_month)] = above_within_below(df[final_month], df['lower_std'], df['upper_std'])
    df.columns.name = 'Monthly {}'.format(event)
    
    # Save to excel
    df.to_excel(tables, sheet_name='Comparative {}'.format(event), index=True)
    

# Step 3 - Volume and proportion of NFA contacts (final month only)

nfa = master[master.year_month == final_month][['agg_contact_source', 'total_contacts', 'contact_only']]
nfa['contacts_nfa_pct'] = nfa['contact_only'] / nfa['total_contacts']

# Save to excel
nfa.to_excel(tables, sheet_name='Contact only', index=False)


# Step 4 - Total contacts, referrals, assessments and lower/upper deviation bounds

total = master[['year_month', 'total_contacts', 'contact_only',
       'led_to_referral', 'led_to_assessment', 'total_referrals',
       'total_assessments']].groupby('year_month').sum()

# Add standard deviation cols
cols = ['total_contacts', 'total_referrals','total_assessments']
for col in cols:
    total['{}_std_lower'.format(col)] = total[col].mean() - total[col].std()
    total['{}_std_upper'.format(col)] = total[col].mean() + total[col].std()

# Save to excel
total.to_excel(tables, sheet_name='Total', index=True)



# Save all tables
tables.save()